In [40]:
import asyncio
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from llm_config import LLMConfig
@dataclass
class Message:
    content: str

In [52]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__(AgentId("simple_agent", "default"), description="Simple agent")
        
    @message_handler
    async def handle_message(self, message: Message ,ctx:MessageContext) -> Message:
        return Message(content=f"this is {self.id.type}--{self.id.key}: you said{message.content}, i disagree with you")


In [42]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime,"simple_agent",lambda:SimpleAgent())


AgentType(type='simple_agent')

In [53]:
runtime.start()

RuntimeError: Runtime is already started

In [54]:

agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Well hi there!"), agent_id)
print(">>>", response.content)

TypeError: RoutedAgent.__init__() got multiple values for argument 'description'

In [45]:
await runtime.stop()
await runtime.close()

In [ ]:
from autogen_core import CancellationToken


class MyLLMAgent(RoutedAgent):
    def __init__(self)->None:
        super().__init__(AgentId("LLMAgent", "default"), description="LLM agent")
        model_client = LLMConfig.model_client
        self._delegate = AssistantAgent(
            agent_id=AgentId("LLMAgent", "default"),
            model_client=model_client,
            system_message="You are a helpful assistant.",
        )
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)


In [55]:
from autogen_core import SingleThreadedAgentRuntime

runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

AgentType(type='LLMAgent')

In [ ]:

response = await runtime.send_message(Message("Hi there!"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

RuntimeError: Runtime is already started